### Fraud User Data collection

In [23]:
import pandas as pd 
from glob import glob
import numpy as np
from collections import Counter 

In [2]:
fraud_users = pd.read_csv('/Users/seongjungkim/PycharmProjects/AnomalyDetection/BLACK LIST/BLACK LIST.csv', 
                         encoding='cp1252')

#### IP_ADDRESS , BLOCKED_IP_ADDRESS 은 모두 fraud ip 임으로 2개의 ip 을 모두 가져온다

In [3]:
fraud_users_ip = pd.concat([fraud_users.IP_ADDRESS, fraud_users.BLOCKED_IP_ADDRESS])


In [4]:
print('# log : {}'.format(len(fraud_users_ip.values)))
fraud_count = Counter(fraud_users_ip.values)
fraud_users_ip = set(fraud_users_ip.values)
print('# Fraud ip : {}'.format(len(fraud_users_ip)))

# log : 26488
# Fraud ip : 8559


In [5]:
# ip 데이터가 [xxx.xxx.xxx.xxx] 이런 형태로 되어 있어 "[" , "]" 을 제거 합니다.
preproc_fraud_ip = [str(line).replace(' ', '').replace(']', '').replace('[', '') for line in list(set(fraud_users_ip))]
preproc_fraud_ip = set(fraud_users_ip)

### Extract All Log IP


In [6]:
"""
Cell : 각 파일별 [계층 구조 상] Feature 개 수 측정
"""

files = glob('./payment-11-logs/payment-11-logs/*')
total_logs = 0
global_ip = []
for file in files:
    # open file 
    f = open(file, encoding='UTF-8')
    
    # read all lines from file 
    lines = f.readlines()

    # total log line count 
    total_logs += len(lines)
    
    # 각 line 별로 Feature 개 수를 파악함. 개 수 파악후 ip 순서대로 을 수집
    for line in lines:
        features = line.split('\t')

        # 각 Log Data 개 수 별로 따로 로그를 저장함. 
        # 아래 셀에서 Test 코드를 작성 하기 위함. 
        if len(features) == 5:
            global_ip.append(features[1])
        elif len(features) == 7:
            global_ip.append(features[2])
        else:
            pass;

    print('File Name : {} # Logs : {}'.format(file, len(lines)))    

print('ip 개 수 {}'.format(len(set(global_ip))))


File Name : ./payment-11-logs/payment-11-logs/payment-202005.txt # Logs : 92397
File Name : ./payment-11-logs/payment-11-logs/payment-202004.txt # Logs : 96514
File Name : ./payment-11-logs/payment-11-logs/payment-201910.txt # Logs : 107906
File Name : ./payment-11-logs/payment-11-logs/payment-202001.txt # Logs : 126764
File Name : ./payment-11-logs/payment-11-logs/payment-201911.txt # Logs : 159671
File Name : ./payment-11-logs/payment-11-logs/payment-201907.txt # Logs : 62621
File Name : ./payment-11-logs/payment-11-logs/payment-202003.txt # Logs : 92450
File Name : ./payment-11-logs/payment-11-logs/payment-202002.txt # Logs : 113469
File Name : ./payment-11-logs/payment-11-logs/payment-201912.txt # Logs : 167541
File Name : ./payment-11-logs/payment-11-logs/payment-201908.txt # Logs : 97529
File Name : ./payment-11-logs/payment-11-logs/payment-201909.txt # Logs : 77239
ip 개 수 85550


### Preprocessing

In [7]:
preproc_all_ip = [line.replace(' ', '').replace(']', '').replace('[', '') for line in list(set(global_ip)) ]
preproc_all_ip = set(preproc_all_ip)

### 모든 로그의 ip 와 Fraud 로그의 ip 가 겹치는 아이디를 파악한다.


In [8]:
len(preproc_fraud_ip & preproc_all_ip)

4698

### 한줄로 핀 데이터 로그를 불러 옵니다. 


In [9]:
data_path = '/Users/seongjungkim/Downloads/isfraud.csv'
df = pd.read_csv(data_path)

/Users/seongjungkim/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,3,6,10,11,13,17,18,19,20,21,22,23,24,25,28,35,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### black list 에 있는 ip 와 match 이 되는 ip 표기 

In [55]:
# ip element 에 있는 '[' ']' 및 공백을 제거 
def ip_preproc(x):
    return x.replace('[','').replace(']','').replace(' ', '')
df.ip = df.loc[:, df.columns == 'ip'].apply(np.vectorize(ip_preproc))

In [56]:
df.isfraud = False
mask = df.ip.isin(fraud_users_ip)

In [60]:
# 겹치는 ip 확인
mask.value_counts()

False    1083286
True      110170
Name: ip, dtype: int64

In [58]:
df.isfraud[mask] = True

/Users/seongjungkim/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### 사기 표시가 된 DataFrame 저장

In [63]:
df.to_csv("merged_fraud_ip.csv", mode='w')